In [11]:
# going_long_bot_dual_tf.py
import time
import ccxt
import numpy as np
import pandas as pd
import talib
import torch
import torch.nn as nn
import joblib

In [12]:
# -------------------------
# CONFIG
# -------------------------
API_KEY = ""      
API_SECRET = ""

symbol = "BTC/USDT"
model_timeframe = "15m"   # model trained on 1-hour candles
price_timeframe = "1s"   # execution monitored on 1-minute candles
limit = 200              
window_size = 10         

FEATURES = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

SAVED_MODEL = "greg_tech_5.pth"
SCALER_FILE = "scaler_15m.pkl"

min_tp_pct = 0.001       
stop_loss_pct = 0.001     
position_fraction = 1  
check_interval_seconds = 1
simulate_only = True     

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [13]:
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]  # last timestep
        out = self.fc(out)
        return out


model = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)
model.load_state_dict(torch.load(SAVED_MODEL))
model.to(device)
model.eval()

scaler = joblib.load(SCALER_FILE)


In [14]:
# -------------------------
# EXCHANGE
# -------------------------
exchange = ccxt.binance({'enableRateLimit': True})


In [15]:
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi


In [16]:
import ta
def compute_indicators(df):
    out = {}
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    sma_window = 3
    out['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)
    out['TSI'] = compute_tsi(df['close'])
    period = 14
    smooth_k = 3
    smooth_d = 3

    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()

    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    
    feat_df = pd.DataFrame(out, index=df.index)
    return feat_df


In [17]:
 #-------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df

In [18]:
# -------------------------
# PREDICTION
# -------------------------
def predict_label_from_window(window_features):
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        pred = torch.argmax(out, dim=1).item()
    return pred  # 0=down, 1=no, 2=up


In [19]:
# -------------------------
# TRADING STATE
# -------------------------
balance = 1000.0
position = 0            
entry_price = None
trade_size = 0.0
current_tp = None
current_sl = None
legs = 0                

def execute_buy(price):
    global balance, position, entry_price, trade_size, current_tp, current_sl, legs
    position = 1
    entry_price = price
    trade_size = balance * position_fraction
    current_sl = entry_price * (1 - stop_loss_pct)
    current_tp = entry_price * (1 + min_tp_pct)
    legs = 0
    print(f"[ENTER LONG] price={price:.2f}, size={trade_size:.2f}, SL={current_sl:.2f}, TP={current_tp:.2f}")

def execute_sell_close(price):
    global balance, position, entry_price, trade_size, current_tp, current_sl
    if position == 1 and entry_price is not None:
        pnl = trade_size * ((price - entry_price) / entry_price)
        balance += pnl
        print(f"[EXIT LONG] price={price:.2f}, P&L={pnl:.2f}, new_balance={balance:.2f}")
    position = 0
    entry_price = None
    current_tp = None
    current_sl = None


In [22]:
# -------------------------
# MAIN LOOP
# -------------------------
print("Starting going-long logic with 1h model + 1m execution. Press Ctrl+C to stop.")
try:
    while True:
        # 1) Model prediction from 1h data
        df_ohlcv = fetch_latest_ohlcv(symbol, model_timeframe, limit)
        feat_df = compute_indicators(df_ohlcv).dropna()
        if len(feat_df) < window_size:
            print("Not enough data. Waiting...")
            time.sleep(10)
            continue
        last_rows = feat_df[FEATURES].values[-window_size:]
        pred = predict_label_from_window(last_rows)

        # 2) Current price from 1m data
        price_df = fetch_latest_ohlcv(symbol, price_timeframe, 2)
        last_price = price_df['close'].values[-1]

        print(f"[{pd.Timestamp.utcnow()}] Predicted: {pred}, Price: {last_price:.2f}, Balance: {balance:.2f}, Position: {position}")

        # Trading logic
        if pred == 2 and position == 0:
            execute_buy(last_price)

        if position == 1:
            price = last_price
            if price <= current_sl:
                print("STOP LOSS HIT")
                execute_sell_close(price)
            elif price >= current_tp:
                legs += 1
                prev_tp = current_tp
                pnl = trade_size * ((prev_tp - entry_price) / entry_price)
                balance += pnl
                print(f"TAKE PROFIT REACHED (leg {legs}): realized {pnl:.2f}, balance now {balance:.2f}")
                entry_price = prev_tp
                current_sl = entry_price * (1 - stop_loss_pct)
                current_tp = entry_price * (1 + min_tp_pct)
                trade_size = balance * position_fraction
                print(f"New leg entered at {entry_price:.2f}, new SL={current_sl:.2f}, next TP={current_tp:.2f}")

        if position == 1 and pred == 0:
            print("MODEL PREDICTS DOWNTREND -> CLOSE POSITION")
            execute_sell_close(last_price)

        time.sleep(check_interval_seconds)

except KeyboardInterrupt:
    print("Stopped by user. Final balance:", balance)

Starting going-long logic with 1h model + 1m execution. Press Ctrl+C to stop.
[2025-10-01 09:32:56.373759+00:00] Predicted: 1, Price: 116234.04, Balance: 1000.00, Position: 0
[2025-10-01 09:32:59.324969+00:00] Predicted: 1, Price: 116234.04, Balance: 1000.00, Position: 0
[2025-10-01 09:33:02.175537+00:00] Predicted: 1, Price: 116234.05, Balance: 1000.00, Position: 0
[2025-10-01 09:33:05.333658+00:00] Predicted: 1, Price: 116234.05, Balance: 1000.00, Position: 0
[2025-10-01 09:33:07.717858+00:00] Predicted: 1, Price: 116234.04, Balance: 1000.00, Position: 0
[2025-10-01 09:33:09.660663+00:00] Predicted: 1, Price: 116234.04, Balance: 1000.00, Position: 0
[2025-10-01 09:33:12.345633+00:00] Predicted: 1, Price: 116234.04, Balance: 1000.00, Position: 0
[2025-10-01 09:33:14.717234+00:00] Predicted: 1, Price: 116234.05, Balance: 1000.00, Position: 0
[2025-10-01 09:33:17.108880+00:00] Predicted: 1, Price: 116234.04, Balance: 1000.00, Position: 0
[2025-10-01 09:33:19.804182+00:00] Predicted: 1, 

RequestTimeout: binance GET https://api.binance.com/api/v3/klines?interval=15m&limit=200&symbol=BTCUSDT